In [ ]:
import sys
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from collections import defaultdict
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class SentimentClassifier(nn.Module):

  def __init__(self):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-cased')
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, 3)
  
  def forward(self, input_ids, attention_mask):
    bert_out = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    pooled_output = bert_out['pooler_output']
    # print(pooled_output)
    output = self.drop(pooled_output)
    prob = F.softmax(self.out(output))
    return prob

In [ ]:
MAX_LEN = 250

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
# model = torch.load('../models/bert_sentiment_model.pt', map_location=torch.device('cpu'))
model = SentimentClassifier()
model.load_state_dict(torch.load('../models/bert_sentiment_model.pt', map_location=torch.device('cpu')))

In [ ]:
def get_sentence_sentiment(s):
    text = s
    encoded_review = tokenizer.encode_plus(
    text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
    )
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)

    output = model(input_ids, attention_mask)
    return int(torch.argmax(output) - 1)


# in this function, we need the scalar, the n (number of rows and columns), and m, the row to apply the scalar to
def generate_matrix(scalar, n, r):
    mat = np.identity(n)
    mat[:,r] *= scalar
    return mat







In [ ]:
# imagine that we have 2 dictionaries
# one dictionary is rid to list of sentences
# one dictionary is rid to tf idf matrix

# iterate through the rids
rids = rid_sid_dict.keys()

rid_feature_dict = {}
for rid in rids:
    # get the sentences in order
    sentences = rid_sid_dict[rid]
    sentiments = map(get_sentence_sentiment, sentences)
    
    # this is the number of rows in the tf idf matrix
    n = len(sentiments)
    
    # create the matrices for sentiments (to be able to multiply through)
    sentiment_matrices = []
    for i in range(0, n):
        sentiment_matrices.append(generate_matrix[sentiments[i], n, i])
        
    tfidf_matrix = rid_tfidf_dict[rid]
    tfidf_matrix_w = tfidf_matrix.copy()
    
    # for each sentiment matrix, multiply it by the tfidf weighted
    for m in sentiment_matrices:
        tfidf_matrix_w = np.matmul(m, tfidf_matrix_w)
        
    # sum over all columns of the  m
    # sum over all columns of the matrix, for both weighted and not weighted
    freq_vec = np.sum(tfidf_matrix, axis=0)
    freq_vec_w = np.sum(tfidf_matrix_w, axis=0)
    
    # apply the mask for params and save both weighted and non weighted vectors for this rid
    rid_feature_dict[rid] = (np.matmul(param_mask, freq_vec.T).T, np.matmul(param_mask, freq_vec_w.T).T)
